# Урок 6
Взаимосвязь величин. Показатели корреляции. Корреляционный анализ. Проверка на нормальность
Домашнее задание

Задача 1

Даны значения величины заработной платы заемщиков банка (salary) и значения их поведенческого кредитного скоринга (scoring):

salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

Используя только встроенные питоновские функции и структуры данных (т.е. без библиотек numpy, pandas и др.) найдите:
1. ковариацию (смещённую и несмещённую) этих двух величин,
2. коэффициент корреляции Пирсона,
3. коэффициент корреляции Кендалла.

Можно затем посчитать те же значения с использованием библиотек, чтобы проверить себя.

In [1]:
salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]
mean_salary = sum(salary) / len(salary)
mean_scoring = sum(scoring) / len(scoring)

In [2]:
# 1. ковариация смещённая
cov_sum = 0
for i in range(len(salary)):
    cov_sum += (salary[i] - mean_salary) * (scoring[i] - mean_scoring)
cov_s = cov_sum / len(salary)
cov_s

9157.84

In [3]:
# ковариация несмещённая
cov_n = cov_sum / (len(salary) - 1)
cov_n

10175.377777777778

In [4]:
def std_find(li: list):
    mean_li = sum(li) / len(li)
    mmean_sqr_sum = 0
    for i in range(len(li)):
        mmean_sqr_sum += (li[i] - mean_li)**2
    # несмещённая оценка дисперсии
    varn_li = mmean_sqr_sum / (len(li) - 1)
    # среднее квадратичное отклонение (несмещённоe)
    return ((varn_li)**(1/2))

In [5]:
# 2. коэффициент корреляции Пирсона
corr = cov_n / (std_find(salary) * std_find(scoring))
corr

0.8874900920739162

In [6]:
# 3. коэффициент корреляции Кендалла
from itertools import combinations
def is_concordant(pair1: tuple, pair2: tuple) -> bool:
    """Являются ли две пары согласованными"""
    return (pair1[0] - pair2[0]) * (pair1[1] - pair2[1]) > 0

In [7]:
P = 0
Q = 0

for pair1, pair2 in combinations(zip(salary, scoring), r=2):
    if is_concordant(pair1, pair2):
        P += 1
    else:
        Q += 1
        
P, Q

(43, 2)

In [8]:
tau = (P - Q) / (P + Q)
tau

0.9111111111111111

In [9]:
import numpy as np
import pandas as pd
from scipy import stats
df = pd.DataFrame({'salary': salary, 'scoring': scoring})

In [10]:
X = df['salary']
Y = df['scoring']

MX = X.mean()
MY = Y.mean()

# ковариация смещённая
cov_s = ((X - MX) * (Y - MY)).sum() / X.shape[0]
cov_s

9157.84

In [11]:
print(np.cov(X, Y, ddof=0))

[[ 3494.64  9157.84]
 [ 9157.84 30468.89]]


In [12]:
# ковариация несмещённая
cov = ((X - MX) * (Y - MY)).sum() / (X.shape[0] - 1)
cov

10175.37777777778

In [13]:
print(np.cov(X, Y, ddof=1))

[[ 3882.93333333 10175.37777778]
 [10175.37777778 33854.32222222]]


In [14]:
# 2. коэффициент корреляции Пирсона
corr = cov / (X.std() * Y.std())
corr

0.8874900920739164

In [15]:
df[['salary', 'scoring']].corr()

,salary,scoring
salary,1.00000,0.88749
scoring,0.88749,1.00000


In [16]:
np.corrcoef(X, Y)

array([[1.        , 0.88749009],
       [0.88749009, 1.        ]])

In [17]:
# 3. коэффициент корреляции Кендалла
pd.DataFrame({'x': X, 'y': Y}).corr(method='kendall')

,x,y
x,1.000000,0.911111
y,0.911111,1.000000


Задача 2

Измерены значения IQ выборки студентов, обучающихся в местных технических вузах:

131, 125, 115, 122, 131, 115, 107, 99, 125, 111

Известно, что в генеральной совокупности IQ распределен нормально. Найдите доверительный интервал для математического ожидания с надежностью 0.95.

In [18]:
iq = np.array([131, 125, 115, 122, 131, 115, 107, 99, 125, 111])
n = iq.shape[0]

mean = iq.mean()
std = iq.std(ddof=1)

n, mean, std

(10, 118.1, 10.54566788359614)

In [19]:
# Поскольку дисперсия нам неизвестна,будем использовать  t-распределение и t-статистику Стьюдента:
p = 0.95
alpha = 1 - p

t = stats.t.ppf(1 - alpha / 2, df=n - 1)
t

2.2621571627409915

In [20]:
# Итак, доверительный интервал:
(mean - t * std / np.sqrt(n), mean + t * std / np.sqrt(n))

(110.55608365158724, 125.64391634841274)

Задача 3

Известно, что рост футболистов в сборной распределён нормально с известной дисперсией 25. На выборке объёма 27 выборочное среднее составило 174.2. Найдите доверительный интервал для математического ожидания с надёжностью 0.95.

Если рассматриваемая случайная величина имеет нормальное распределение, то доверительный интервал для математического ожидания можно построить с помощью квантилей стандартного распределения (если дисперсия известна).

$$T(X) = \dfrac{\overline{X} - a}{\sigma / \sqrt{n}},$$
где $\sigma$ — известный параметр среднего квадратического отклонения. Статистика $T$ имеет стандартное распределение. Допустим, нас интересует симметричный доверительный интервал с уровнем доверия $p = 1 - \alpha$. Тогда доверительный интервал для $T$ можно получить с помощью соответствующих квантилей:
$$P(- t_{1 - \alpha / 2} \leq T \leq t_{1 - \alpha / 2}) = p.$$

Преобразуем выражения в скобках и получим доверительный интервал для параметра $a$:
$$
P \left(
    \overline{X} - t_{1 - \alpha / 2} \cdot \dfrac{\sigma}{\sqrt{n}}
    \leq a \leq 
    \overline{X} + t_{1 - \alpha / 2} \cdot \dfrac{\sigma}{\sqrt{n}}
\right) = p.
$$

In [21]:
mean = 174.2
n = 27
p = 0.95
scale = np.sqrt(25)
p = 0.95
alpha = 1 - p

t = stats.norm.ppf(1 - alpha/2)
t

1.959963984540054

In [22]:
# Итак, доверительный интервал:
(mean - t * scale / np.sqrt(n), mean + t * scale / np.sqrt(n))

(172.3140237765397, 176.08597622346028)

Задача 4

Выберите тему для проектной работы по курсу Теории вероятностей и математической статистики и напишите ее в комментарии к Практическому заданию.

In [ ]:
# Есть данные по ценам квадратного метра загородки в населенных пунктах МО 
# с авито, добавлено расстояние от МКАД и численность населения:
# https://docs.google.com/spreadsheets/d/1n_LZWqz3hy0FDHoPmEriTgJO27tDDT2VnoH6RLmyhUg/edit?usp=sharing
# Хочу проанализировать:
# - зависимость цен от километража и численности населения,
# - есть ли выраженные сезонные колебания цен,
# - сделать прогноз на апрель
# Ok?